In [4]:
from scapy.all import rdpcap, IP, TCP, UDP, Raw, IPv6

In [2]:
# Ensure the PCAP file exists before reading
pcap_file = "0.pcap"
try:
    packets = rdpcap(pcap_file)
except FileNotFoundError:
    print(f"File {pcap_file} not found")

In [6]:
# ims_ip = "14.139.98.79"
ims_ip = "64:ff9b::e8b:624f"

In [ ]:
# ims_ip = "14.139.98.79"  # As per nslookup.com
ims_ip = "64:ff9b::e8b:624f"  # As per nslookup.com
unique_connections = set()
courses = set()
data_transfer = 0
superuser_count = 0
dt_list = []

for pkt in packets:
    if IP in pkt:
        src_ip = pkt[IP].src
        dst_ip = pkt[IP].dst

        if dst_ip == ims_ip and (TCP in pkt or UDP in pkt):
            unique_connections.add(src_ip)
        
        if dst_ip == ims_ip and Raw in pkt:
            payload = pkt[Raw].load.decode(errors='ignore')
            if "course" in payload.lower():
                courses.add(payload)
        
        # if (TCP in pkt and (pkt[TCP].sport == 4321 or pkt[TCP].dport == 4321)) or (UDP in pkt and (pkt[UDP].sport == 4321 or pkt[UDP].dport == 4321)):
        protocol = TCP if TCP in pkt else UDP if UDP in pkt else None
        if protocol and (pkt[protocol].sport == 4321 or pkt[protocol].dport == 4321):
            data_transfer += len(pkt)
            dt_list.append(pkt)
        
        if Raw in pkt:
            payload = pkt[Raw].load.decode(errors='ignore')
            superuser_count += payload.lower().count("superuser")

In [12]:
port_packets, ims_packets = [], []
for packet in packets:
    if IP in packet:
        src_ip = packet[IP].src
        dst_ip = packet[IP].dst
        if dst_ip == ims_ip:
            ims_packets.append(packet)
        
        protocol = TCP if TCP in packet else UDP if UDP in packet else None
        if protocol and (packet[protocol].sport == 4321 or packet[protocol].dport == 4321):
            port_packets.append(packet)

In [13]:
port_packets

[<Ether  dst=2c:41:38:1d:e9:40 src=00:90:7f:3e:02:d0 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=103 id=17453 flags= frag=0 ttl=126 proto=udp chksum=0x9996 src=172.16.128.169 dst=172.16.133.248 |<UDP  sport=rwhois dport=snmp len=83 chksum=0x9082 |<SNMP  version='v1' 0x0 <ASN1_INTEGER[0]> community=<ASN1_STRING[b'public']> PDU=<SNMPget  id=0x491c2f5 <ASN1_INTEGER[76661493]> error='no_error' 0x0 <ASN1_INTEGER[0]> error_index=0x0 <ASN1_INTEGER[0]> varbindlist=[<SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.2.2.1.10.18']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>, <SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.2.2.1.16.18']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>, <SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.1.3.0']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>] |> |>>>>,
 <Ether  dst=00:90:7f:3e:02:d0 src=2c:41:38:1d:e9:40 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=115 id=62980 flags= fra

In [14]:
sum([len(packet) for packet in port_packets])

246

In [43]:
port_packets[0].show2()

###[ Ethernet ]###
  dst       = 2c:41:38:1d:e9:40
  src       = 00:90:7f:3e:02:d0
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 103
     id        = 17453
     flags     = 
     frag      = 0
     ttl       = 126
     proto     = udp
     chksum    = 0x9996
     src       = 172.16.128.169
     dst       = 172.16.133.248
     \options   \
###[ UDP ]###
        sport     = rwhois
        dport     = snmp
        len       = 83
        chksum    = 0x9082
###[ SNMP ]###
           version   = 'v1' 0x0 <ASN1_INTEGER[0]>
           community = <ASN1_STRING[b'public']>
           \PDU       \
            |###[ SNMPget ]###
            |  id        = 0x491c2f5 <ASN1_INTEGER[76661493]>
            |  error     = 'no_error' 0x0 <ASN1_INTEGER[0]>
            |  error_index= 0x0 <ASN1_INTEGER[0]>
            |  \varbindlist\
            |   |###[ SNMPvarbind ]###
            |   |  oid       = <ASN1_OID['.1.3.6.1.2.1.2.2.1.10.18']>
   

In [22]:
# Print Results
print(f"Q1: Unique connections to IMS server: {len(unique_connections)}")
print(f"Q2: Registered Courses: {courses}")
print(f"Q3: Total data transferred on port 4321: {data_transfer} bytes")
print(f"Q4: Total number of SuperUsers: {superuser_count}")

Q1: Unique connections to IMS server: 0
Q2: Registered Courses: set()
Q3: Total data transferred on port 4321: 246 bytes
Q4: Total number of SuperUsers: 0


In [23]:
dt_list

[<Ether  dst=2c:41:38:1d:e9:40 src=00:90:7f:3e:02:d0 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=103 id=17453 flags= frag=0 ttl=126 proto=udp chksum=0x9996 src=172.16.128.169 dst=172.16.133.248 |<UDP  sport=4321 dport=snmp len=83 chksum=0x9082 |<SNMP  version='v1' 0x0 <ASN1_INTEGER[0]> community=<ASN1_STRING[b'public']> PDU=<SNMPget  id=0x491c2f5 <ASN1_INTEGER[76661493]> error='no_error' 0x0 <ASN1_INTEGER[0]> error_index=0x0 <ASN1_INTEGER[0]> varbindlist=[<SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.2.2.1.10.18']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>, <SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.2.2.1.16.18']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>, <SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.1.3.0']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>] |> |>>>>,
 <Ether  dst=00:90:7f:3e:02:d0 src=2c:41:38:1d:e9:40 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=115 id=62980 flags= frag=

In [24]:
for pkt in dt_list:
    pkt.show()

###[ Ethernet ]###
  dst       = 2c:41:38:1d:e9:40
  src       = 00:90:7f:3e:02:d0
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 103
     id        = 17453
     flags     = 
     frag      = 0
     ttl       = 126
     proto     = udp
     chksum    = 0x9996
     src       = 172.16.128.169
     dst       = 172.16.133.248
     \options   \
###[ UDP ]###
        sport     = 4321
        dport     = snmp
        len       = 83
        chksum    = 0x9082
###[ SNMP ]###
           version   = 'v1' 0x0 <ASN1_INTEGER[0]>
           community = <ASN1_STRING[b'public']>
           \PDU       \
            |###[ SNMPget ]###
            |  id        = 0x491c2f5 <ASN1_INTEGER[76661493]>
            |  error     = 'no_error' 0x0 <ASN1_INTEGER[0]>
            |  error_index= 0x0 <ASN1_INTEGER[0]>
            |  \varbindlist\
            |   |###[ SNMPvarbind ]###
            |   |  oid       = <ASN1_OID['.1.3.6.1.2.1.2.2.1.10.18']>
     